In [27]:
# data_collection_scraper
import requests
from bs4 import BeautifulSoup
import time
import random
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import urllib
import pdfkit
import os

In [29]:
# Anti-blocking: List of User-Agents to rotate
USER_AGENTS = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.121 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:79.0) Gecko/20100101 Firefox/79.0",
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.121 Safari/537.36"
]


In [31]:
# Set up WebDriver options
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument("--disable-extensions")
options.add_argument("--disable-popup-blocking")
options.add_argument("--disable-infobars")
options.add_argument(f"user-agent={random.choice(USER_AGENTS)}")

In [33]:
# Initialize WebDriver
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

In [35]:
if(not os.path.isfile('results3.csv')):
    df = pd.DataFrame(pd.DataFrame(columns=['Ticker', 'Entity Name', 'Form Name', 'File Date''URL']))
    df.to_csv('results3.csv', mode='a', index=False)

In [37]:
# Initate output data fram
output = pd.DataFrame()

# load firm list containing tickers 
firmList = pd.read_csv("firmList_full.csv")

In [47]:
firm_name = firmList['firm'][0].replace('THE', '').replace('LIMITED', '').replace(' ', '%2520').replace('&', '%2526')
ticker = firmList['ticker'][0]

In [49]:
print(firm_name)

General%2520Motors%2520Co


In [51]:
print(ticker)

GM


In [53]:
# Example processing of a single firm
firm_name_raw = firmList['firm'][0]
firm_name = urllib.parse.quote(firm_name_raw.replace('THE', '').replace('LIMITED', ''))
ticker = firmList['ticker'][0]

print("Formatted Firm Name for URL:", firm_name)  # URL encoded firm name
print("Ticker:", ticker)

Formatted Firm Name for URL: General%20Motors%20Co
Ticker: GM


In [55]:
url = 'https://www.sec.gov/edgar/search/#/q=5.02'+'&dateRange=custom&'+ 'entityName='+ ticker +'&startdt=2015-01-01&enddt=2023-02-24&filter_forms=8-K'

In [57]:
print(url)

https://www.sec.gov/edgar/search/#/q=5.02&dateRange=custom&entityName=GM&startdt=2015-01-01&enddt=2023-02-24&filter_forms=8-K


In [59]:
driver.get(url)

In [61]:
html = driver.page_source
soup = BeautifulSoup(html, 'lxml')
# print(soup)

In [63]:
part= soup.find('td', class_='cik d-none').text.replace('CIK ', '').lstrip('0')
links = soup.find_all('a', class_='preview-file')
# print(part)
# print(links)

In [65]:
form_names = []
file_links = []
file_dates = []
entity_names = []

In [67]:
for i in range(len(links)):
    new_link = 'https://www.sec.gov/Archives/edgar/data/'+part+'/'+links[i].attrs['data-adsh'].replace('-', '')+'/'+links[i].attrs['data-file-name']
    file_links.append(new_link)

In [69]:
# print(file_links)

In [71]:
firmList.shape

(41, 2)

In [73]:
# look through firms by ticker
for i in range(0, firmList.shape[0]):
    firm_name = firmList['firm'][i].replace('THE', '').replace('LIMITED', '').replace(' ', '%2520').replace('&', '%2526')
    ticker = firmList['ticker'][i]

    # assemble page link
    url = 'https://www.sec.gov/edgar/search/#/q=5.02'+'&dateRange=custom&'+ 'entityName='+ ticker + '%252C%2520'+firm_name + '&startdt=2015-01-01&enddt=2023-02-24'
    print(url)
    driver.get(url)
    time.sleep(2)  # Wait for the new results to load 
    
    driver.refresh()
    time.sleep(1)  
    
    # Define the number of times to scroll
    scroll_count = 5
    
    # Simulate continuous scrolling using JavaScript
    for _ in range(scroll_count):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(4)  # Wait for the new results to load 
    
    # get page source
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')


    # initiate data frame
    form_names = []
    file_links = []
    file_dates = []
    entity_names = []

    form_names = [i.text for i in soup.find_all('a', class_='preview-file')]
    file_dates = [i.text for i in soup.find_all('td', class_='filed')]
    entity_names = [i.text for i in soup.find_all('td', class_='entity-name')]

    # download directory
    directory = (ticker)
    if not os.path.exists(directory):
        os.makedirs(directory)


    no_results_element = soup.find('div', class_='col-sm-12', role='alert', style='display: block;')
    if no_results_element and no_results_element.find('h4', class_='text-center') and 'No results found for your search!' in no_results_element.text:
        print("No results found for your search.")
    else:
        # Check for the 'td' element with class 'cik d-none'
        cik_element = soup.find('td', class_='cik d-none')
        if cik_element:
            part = cik_element.text.replace('CIK ', '').lstrip('0')
            print(f"CIK Found: {part}")        
    
            part= soup.find('td', class_='cik d-none').text.replace('CIK ', '').lstrip('0')
            links = soup.find_all('a', class_='preview-file')
            for i in range(len(links)):
                new_link = 'https://www.sec.gov/Archives/edgar/data/'+part+'/'+links[i].attrs['data-adsh'].replace('-', '')+'/'+links[i].attrs['data-file-name']
                file_links.append(new_link)
        
            dict = {'Ticker': [ticker]*len(form_names), 'Entity Name': entity_names, 'Form Name': form_names, 'File Date':file_dates, 'URL': file_links} 
            df = pd.DataFrame(dict)
            df = df[df['Form Name'].str.contains('8-K')]
            df = df[~df['Form Name'].str.contains('EX-')]
        
            # dowloand files
            for url, file_date in zip(df['URL'], df['File Date']):
                try:
                    if 'htm' in url:
                        # Convert HTML to PDF using pdfkit
                        pdf_file_path = os.path.join(directory, f'{file_date}.pdf')
                        pdfkit.from_url(url, pdf_file_path)
                    elif 'pdf' in url:
                        # Directly download PDF files
                        pdf_file_path = os.path.join(directory, f'{file_date}.pdf')
                        headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36'}
                        response = requests.get(url, headers = headers, stream = True, allow_redirects=True)
                        if response.status_code == 200:
                            with open(pdf_file_path, 'wb') as pdf_file:
                                pdf_file.write(response.content)
                        else:
                            print(f"Failed to download file from {url}")
                    else:
                        print(f"Unsupported file format for URL: {url}")
                except Exception as e:
                    print(f"Error processing URL: {url}")
                    print(f"Error details: {str(e)}")
        
            df.to_csv('results3.csv', mode='a', header=False, index=False)

https://www.sec.gov/edgar/search/#/q=5.02&dateRange=custom&entityName=GM%252C%2520General%2520Motors%2520Co&startdt=2015-01-01&enddt=2023-02-24
CIK Found: 1467858
https://www.sec.gov/edgar/search/#/q=5.02&dateRange=custom&entityName=AEFSF%252C%2520Aeffe%2520SpA&startdt=2015-01-01&enddt=2023-02-24
No results found for your search.
https://www.sec.gov/edgar/search/#/q=5.02&dateRange=custom&entityName=ARGGY%252C%2520Aston%2520Martin%2520Lagonda%2520Global%2520Holdings%2520ADR%2520Representing%2520Ord%2520Shs&startdt=2015-01-01&enddt=2023-02-24
No results found for your search.
https://www.sec.gov/edgar/search/#/q=5.02&dateRange=custom&entityName=AMGDF%252C%2520Aston%2520Martin%2520Lagonda%2520Global%2520Holdings%2520PLC&startdt=2015-01-01&enddt=2023-02-24
No results found for your search.
https://www.sec.gov/edgar/search/#/q=5.02&dateRange=custom&entityName=BMWYY%252C%2520Bayerische%2520Motoren%2520Werke%2520ADR&startdt=2015-01-01&enddt=2023-02-24
No results found for your search.
https:/

In [ ]:
# initiate data frame
form_names = []
file_links = []
file_dates = []
entity_names = []

In [ ]:
form_names = [i.text for i in soup.find_all('a', class_='preview-file')]
file_dates = [i.text for i in soup.find_all('td', class_='filed')]
entity_names = [i.text for i in soup.find_all('td', class_='entity-name')]

In [ ]:
# download directory
directory = (ticker)
if not os.path.exists(directory):
    os.makedirs(directory)

In [ ]:
dict = {'Ticker': [ticker]*len(form_names), 'Entity Name': entity_names, 'Form Name': form_names, 'File Date':file_dates, 'URL': file_links} 
df['Form Name'] = df['Form Name'].astype(str)
df = df[df['Form Name'].str.contains('8-K', na=False)]
df = df[~df['Form Name'].str.contains('EX-', na=False)]

In [ ]:
# dowloand files
for url, file_date in zip(df['URL'], df['File Date']):
    try:
        if 'htm' in url:
            # Convert HTML to PDF using pdfkit
            pdf_file_path = os.path.join(directory, f'{file_date}.pdf')
            pdfkit.from_url(url, pdf_file_path)
        elif 'pdf' in url:
            # Directly download PDF files
            pdf_file_path = os.path.join(directory, f'{file_date}.pdf')
            headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.114 Safari/537.36'}
            response = requests.get(url, headers = headers, stream = True, allow_redirects=True)
            if response.status_code == 200:
                with open(pdf_file_path, 'wb') as pdf_file:
                    pdf_file.write(response.content)
            else:
                print(f"Failed to download file from {url}")
        else:
            print(f"Unsupported file format for URL: {url}")
    except Exception as e:
        print(f"Error processing URL: {url}")
        print(f"Error details: {str(e)}")

In [ ]:
df.to_csv('results3.csv', mode='a', header=False, index=False)